In [3]:
# Install necessary libraries
# !pip install torch torchvision torchaudio pandas scikit-learn

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
df_children_books_final = pd.read_csv('df_children_books_final.csv')
df_children_books_final_encoded = pd.read_csv('df_children_books_final_encoded.csv')
df_interactions_final_merged = pd.read_csv('df_interactions_final_merged.csv')

/var/folders/lv/4p2_jtmx6rzdf0lvp89413tc0000gn/T/ipykernel_77548/2830472867.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_children_books_final_encoded = pd.read_csv('df_children_books_final_encoded.csv')


In [8]:
df_children_books_final.head(3)

,isbn,isbn13,book_id,title,authors,num_pages,language_code,description,format,is_ebook,popular_shelves,average_rating,ratings_count,text_reviews_count,isbn_combined,book_length,top_popular_shelves,author_ids,shelf_names
0,1599150603,9781599150604,287141,The Aeneid for Boys and Girls,"[{'author_id': '3041852', 'role': ''}]",162.0,NaN,"Relates in vigorous prose the tale of Aeneas, ...",Physical,False,"[{'count': '56', 'name': 'to-read'}, {'count':...",4.13,46,7,1599150603,medium,"[{'count': '4', 'name': 'history'}, {'count': ...",['3041852'],"['history', 'classics', 'level-4to5']"
1,1934876569,9781934876565,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,"[{'author_id': '19158', 'role': ''}]",216.0,NaN,"To Kara's astonishment, she discovers that a p...",Physical,False,"[{'count': '515', 'name': 'to-read'}, {'count'...",4.22,98,6,1934876569,medium,"[{'count': '9', 'name': 'favorites'}, {'count'...",['19158'],"['favorites', 'magic', 'avalon']"
2,0590417010,9780590417013,89378,Dog Heaven,"[{'author_id': '5411', 'role': ''}]",40.0,en,In Newbery Medalist Cynthia Rylant's classic b...,Physical,False,"[{'count': '450', 'name': 'to-read'}, {'count'...",4.43,1331,193,0590417010,short,"[{'count': '8', 'name': 'pets'}, {'count': '8'...",['5411'],"['pets', 'children-s-lit', 'fiction']"


In [7]:
df_children_books_final_encoded.head(3)

,isbn,isbn13,book_id,title,authors,num_pages,description,is_ebook,popular_shelves,average_rating,...,lang_tha,lang_tur,lang_ukr,lang_vie,lang_vls,lang_yid,lang_zho,length_long,length_medium,length_short
0,1599150603,9781599150604,287141,The Aeneid for Boys and Girls,"[{'author_id': '3041852', 'role': ''}]",162.0,"Relates in vigorous prose the tale of Aeneas, ...",False,"[{'count': '56', 'name': 'to-read'}, {'count':...",4.13,...,False,False,False,False,False,False,False,False,True,False
1,1934876569,9781934876565,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,"[{'author_id': '19158', 'role': ''}]",216.0,"To Kara's astonishment, she discovers that a p...",False,"[{'count': '515', 'name': 'to-read'}, {'count'...",4.22,...,False,False,False,False,False,False,False,False,True,False
2,0590417010,9780590417013,89378,Dog Heaven,"[{'author_id': '5411', 'role': ''}]",40.0,In Newbery Medalist Cynthia Rylant's classic b...,False,"[{'count': '450', 'name': 'to-read'}, {'count'...",4.43,...,False,False,False,False,False,False,False,False,False,True


In [9]:
df_interactions_final_merged.head(3)

,user_id,book_id,review_id,rating,review_text_incomplete,date_added,n_votes,review_age,processed_review,sentiment
0,8842281e1d1347389f2ab93d60773d4d,23310161,f4b4b050f4be00e9283c92a814af2670,4,Fun sequel to the original.,2015-11-17 19:37:35+00:00,7.0,3248,fun sequel original,0.680800
1,8842281e1d1347389f2ab93d60773d4d,18296097,bc9cff98f54be3b2b8c5b34598a7546c,5,NaN,2015-09-21 15:16:57+00:00,NaN,3305,NaN,0.574139
2,8842281e1d1347389f2ab93d60773d4d,817720,75fd46041466ceb406b7fd69b089b9c5,5,NaN,2015-05-21 04:29:23+00:00,NaN,3429,NaN,0.574139


In [11]:
# Preprocessing
userid_encoder = LabelEncoder()
bookid_encoder = LabelEncoder()

df_interactions_final_merged['user_id_encoded'] = userid_encoder.fit_transform(df_interactions_final_merged['user_id'])
df_interactions_final_merged['book_id_encoded'] = bookid_encoder.fit_transform(df_interactions_final_merged['book_id'])

In [12]:
# Split data into train and test sets
train_data = []
test_data = []

In [13]:
# Group the data by 'user_id' and split into train/test sets
for user_id, group in df_interactions_final_merged.groupby('user_id'):
    if len(group) > 1:
        # All interactions except the last one go to training
        train_data.append(group.iloc[:-1])
        # The last interaction goes to testing
        test_data.append(group.iloc[-1:])
    else:
        # If there's only one interaction, add it to the training set (or adjust logic as needed)
        train_data.append(group)

# Convert the lists to DataFrames
train_data_A = pd.concat(train_data, ignore_index=True)
test_data_A = pd.concat(test_data, ignore_index=True)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107c5bfa0>>
Traceback (most recent call last):
  File "/Users/terence/bt4222-books/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
# Define the NCF model with GMF and MLP
class NCF_A(nn.Module):
    def __init__(self, num_users, num_books, embedding_dim):
        super(NCF_A, self).__init__()
        # GMF Components
        self.user_embeddings_gmf = nn.Embedding(num_users, embedding_dim)
        self.book_embeddings_gmf = nn.Embedding(num_books, embedding_dim)

        # MLP Components
        self.user_embeddings_mlp = nn.Embedding(num_users, embedding_dim)
        self.book_embeddings_mlp = nn.Embedding(num_books, embedding_dim)

        self.fc1_mlp = nn.Linear(2 * embedding_dim, 128)
        self.fc2_mlp = nn.Linear(128, 64)

        # Final layers
        self.fc1_combined = nn.Linear(embedding_dim + 64, 128)
        self.fc2_combined = nn.Linear(128, 1)

    def forward(self, user_id, book_id):
        # GMF
        user_emb_gmf = self.user_embeddings_gmf(user_id)
        book_emb_gmf = self.book_embeddings_gmf(book_id)
        gmf_output = user_emb_gmf * book_emb_gmf

        # MLP
        user_emb_mlp = self.user_embeddings_mlp(user_id)
        book_emb_mlp = self.book_embeddings_mlp(book_id)
        mlp_input = torch.cat([user_emb_mlp, book_emb_mlp], dim=-1)
        mlp_output = torch.relu(self.fc1_mlp(mlp_input))
        mlp_output = torch.relu(self.fc2_mlp(mlp_output))

        # Combine GMF and MLP outputs
        combined_input = torch.cat([gmf_output, mlp_output], dim=-1)
        combined_output = torch.relu(self.fc1_combined(combined_input))
        combined_output = self.fc2_combined(combined_output) * 4 + 1  # Scale to [1, 5]

        return combined_output.squeeze()